## Approach 2 for clustering similar test steps in natural language

* Text embedding technique: BERT
* Text similarity: Word Mover's Distance (WMD)
* Clustering techniques: Hierarchical agglomerative clustering and K-means

In [1]:
# Import necessary libraries
import os
import gc
import pandas as pd
import numpy as np
import math
import statistics as st
import re
import string
import time
import matplotlib.pyplot as plt

# For word frequency
from collections import defaultdict

# ML libraries
from sklearn.cluster import AgglomerativeClustering
from sklearn.cluster import KMeans
import scipy.cluster.hierarchy as sch
import gensim
from gensim.models import Word2Vec, Phrases, KeyedVectors
from gensim.corpora.dictionary import Dictionary
import torch
from transformers import BertTokenizer, BertModel

# NLP libraries
import nltk
from nltk.corpus import stopwords 
from nltk.tokenize import RegexpTokenizer, word_tokenize, TweetTokenizer
from nltk.stem import WordNetLemmatizer 

# To be used with hierarchical clustering
from joblib import Memory

# To save models
import pickle

In [ ]:
# Download wordnet data through nltk
nltk.download('wordnet')

### Data preprocessing functions

In [6]:
# Function to compute number of unique words in df
def get_number_unique_words(df):
    words_list = list()
    test_steps = list(df["Steps"])
    for step in test_steps:
        for word in step:
            words_list.append(word)
    number_unique_words = len(set(words_list))
    return number_unique_words

In [7]:
# Function to compute number of unique words in df ('test case name' field)
def get_number_unique_words_name(df):
    words_list = list()
    test_names = list(df["Case_Name"])
    for name in test_names:
        for word in name:
            words_list.append(word)
    number_unique_words = len(set(words_list))
    return number_unique_words

In [8]:
# Function to get list of words that occur less than a certain number of times
def get_word_frequency(df):
    words_list = list()
    test_steps = list(df["Steps"])
    for step in test_steps:
        for word in step:
            words_list.append(word)
    unique_words_list = set(words_list)
    word_occurrence_dict = {}
    for each_word in unique_words_list:
        word_occurrence_dict[each_word] = 0

    for step in test_steps:
        for word in step:
            word_occurrence_dict[word] += 1
            
    ten_times_occurrence_words = list()
    # get list of words that occur only once
    for word, occurrence in word_occurrence_dict.items():
        if occurrence < 2:
            ten_times_occurrence_words.append(word)

    return ten_times_occurrence_words

In [9]:
# Function to get list of words that occur less than a certain number of times ('test case name' field)
def get_word_frequency_name(df):
    words_list = list()
    test_names = list(df["Case_Name"])
    for name in test_names:
        for word in name:
            words_list.append(word)
    unique_words_list = set(words_list)
    word_occurrence_dict = {}
    for each_word in unique_words_list:
        word_occurrence_dict[each_word] = 0

    for name in test_names:
        for word in name:
            word_occurrence_dict[word] += 1
            
    ten_times_occurrence_words = list()
    # get list of words that occur only once
    for word, occurrence in word_occurrence_dict.items():
        if occurrence < 2:
            ten_times_occurrence_words.append(word)

    return ten_times_occurrence_words

In [10]:
# Function to remove problematic/mispelled words from vocabulary
def remove_problematic_words(df):
    number_unique_words = get_number_unique_words(df)
    print("Number of unique words across all test steps: ", number_unique_words)
    
    # load file with problematic words that exist in the test data
    problematic_words = open('word2vec_vocab_problematic.txt', 'r')
    problematic_words_list = list()
    for word in problematic_words:
        problematic_words_list.append(word.lstrip().rstrip())
    
    for index, row in df.iterrows():
        step = row["Steps"]
        df.loc[index]["Steps"] = [elem for elem in step if not elem in problematic_words_list]
        
    number_unique_words = get_number_unique_words(df)
    print("Number of unique words across all test steps after removing problematic words: ", number_unique_words)

In [11]:
# function to fix problematic/mispelled words from vocabulary
def fix_problematic_words(df):
    number_unique_words = get_number_unique_words(df)
    print("Number of unique words across all test steps: ", number_unique_words)
    
    # load file with problematic words that exist in the test data
    problematic_words = open('word2vec_vocab_to_fix.txt', 'r')
    problematic_words_dict = {}
    for line in problematic_words:
        full_line = line.split(':')
        try:
            problematic_words_dict[full_line[0]] = [x.replace('\n', '') for x in full_line[1].split(',')]
        except:
            problematic_words_dict[full_line[0]] = full_line[1].replace('\n', '')
    
    for index, row in df.iterrows():
        step = row["Steps"]
        modified_step = list()
        for word in step:
            if word in problematic_words_dict:
                modified_step.extend(problematic_words_dict[word])
            else:
                modified_step.append(word)
        df.loc[index]["Steps"] = modified_step 
        
    number_unique_words = get_number_unique_words(df)
    print("Number of unique words across all test steps after fixing problematic words: ", number_unique_words)

In [8]:
def preprocess_clean_data(df):

    # Preprocessing and clean test steps
    print("Cleaning test step field...")
    
    df["Steps"] = df["Steps"].apply(lambda x: re.sub(r'http\S+', 'URL', x))
    df["Steps"] = df["Steps"].apply(lambda x: re.sub('\/[\w-]*', '', x))
    df["Steps"] = df["Steps"].apply(lambda x: re.sub(r'\{[^)]*\}', '', x))

    # lowercase the step descriptions
    df["Steps"] = df["Steps"].apply(lambda x: x.lower())

    # remove digits and words with digits
    df["Steps"] = df["Steps"].apply(lambda x: re.sub('\w*\d\w*','', x))

    # remove punctuations
    df["Steps"] = df["Steps"].apply(lambda x: re.sub('[%s]' % re.escape(string.punctuation), ' ', x))

    # remove extra spaces
    df["Steps"] = df["Steps"].apply(lambda x: re.sub(' +',' ',x))

    # tokenization
    df["Steps"] = df["Steps"].apply(lambda x: TweetTokenizer().tokenize(x))
    number_unique_words = get_number_unique_words(df)
    print("Number of unique words across all test steps: ", number_unique_words)

    remove_problematic_words(df)
    fix_problematic_words(df)
    
    # stopword removal
    stop_words = set(stopwords.words('english'))
    df["Steps"] = df["Steps"].apply(lambda x: [w for w in x if not w in stop_words])
    number_unique_words = get_number_unique_words(df)
    print("Number of unique words in test steps after stopword removal: ", number_unique_words)

    # lemmatization
    lemmatizer = WordNetLemmatizer() 
    df["Steps"] = df["Steps"].apply(lambda x: [lemmatizer.lemmatize(w) for w in x])
    number_unique_words = get_number_unique_words(df)
    print("Number of unique words in test steps after lemmatization: ", number_unique_words)

    # remove words that occur less than 10 times
    ten_times_occurrence_words = get_word_frequency(df)
    print("Number of words that occurred less than 10 times in test steps: ", len(ten_times_occurrence_words))

    for index, row in df.iterrows():
        current_test_step = row["Steps"]
        list_words_to_remove = list()
        for word in current_test_step:
            if word in ten_times_occurrence_words:
                list_words_to_remove.append(word)

        test_steps_df.loc[index]["Steps"] = [elem for elem in current_test_step if not elem in list_words_to_remove]

    print("Dataset size after preprocessing: " , df.shape)

### Read and preprocess files with test cases and build dataframe

In [20]:
# Get data directory and list of xlsx files
current_dir = os.getcwd() 
parent_dir = os.path.dirname(current_dir) + "\\filtered_data\\"
xlsxfiles = [os.path.join(root, name)
             for root, dirs, files in os.walk(parent_dir)
             for name in files
             if name.endswith((".xlsx"))]

In [21]:
# Declare pandas df to be populated
column_names = ["Type", "Key", "Case_Name", "Step_ID", "Steps"]
test_steps_df = pd.DataFrame(columns = column_names)

# Index to add data to the df
index_to_add = 0

print("Reading input data...")   
for test_file in xlsxfiles:
    # load data and iterate through it
    test_data_df = pd.read_excel(test_file)
    for index, row in test_data_df.iterrows():
        current_type = row["Type"]
        current_key = row["Key"]
        current_name = row["Case_Name"]
        current_step_id = row["Step_ID"]
        current_steps = row["Steps"]
        test_steps_df.loc[index_to_add] = [current_type, current_key, current_name, current_step_id, current_steps]
        index_to_add += 1

print("Done!")
print("Shape of data => ", test_steps_df.shape)

Reading input data...
Done!
Shape of data =>  (15668, 5)


In [22]:
# Call preprocessing function
preprocess_clean_data(test_steps_df)

Cleaning test case name field...
Dataset size before preprocessing:  (15668, 5)
Number of unique words across all test names:  1519
Number of unique words in test names after stopword removal:  1447
Number of words that occurred only once in test case names:  164
Number of unique words in test names in the end:  1138
Dataset size after preprocessing:  (15668, 5)


### Define functions to get BERT embeddings

In [14]:
def bert_model_init():
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    
    # Load pre-trained model (weights)
    bert_model = BertModel.from_pretrained('bert-base-uncased',
                                  output_hidden_states = True, # Whether the model returns all hidden-states.
                                  )

    # Put the model in "evaluation" mode, meaning feed-forward operation.
    bert_model.eval()
    return [tokenizer, bert_model]

In [15]:
def get_bert_embedding_sentence_words(bert_model, bert_tokenizer, original_sentence_1, cleaned_sentence_1):
    sentence1_embedd = list()
    
    ### Embeddings for sentence 1
    # Tokenize all of the sentences and map the tokens to thier word IDs
    input_ids = []
    attention_masks = []
    
    # Add special tokens
    marked_text = "[CLS] " + original_sentence_1 + " [SEP]"

    # Split the sentence into tokens
    tokenized_text = bert_tokenizer.tokenize(marked_text)

    # Map the token strings to their vocabulary indeces
    indexed_tokens = bert_tokenizer.convert_tokens_to_ids(tokenized_text)

    # Display the words with their indeces (uncomment to see)
#     for tup in zip(tokenized_text, indexed_tokens):
#         print('{:<12} {:>6,}'.format(tup[0], tup[1]))

    # Convert indexed_tokens to tensor
    tokens_tensor = torch.tensor([indexed_tokens])
    
    with torch.no_grad():
        outputs = bert_model(tokens_tensor)
        hidden_states_list = outputs[2]
    
    hidden_states = torch.stack(hidden_states_list, dim=0)

    # Remove dimension 1, the "batches" (because we only have 1 sentence)
    hidden_states = torch.squeeze(hidden_states, dim=1)
    
    # Sum 4 last layers
    hidden_states_last_4 = hidden_states[-4:]
    
    # Start index with 1 because index zero of tokenized_text is always [CLS]
    token_index = 1 
    for search_word in cleaned_sentence_1:
        for token_i in range(token_index, len(tokenized_text)-1):
            found = False
            if search_word == tokenized_text[token_i]:

                # Sum 4 last layers
                vec = torch.sum(hidden_states_last_4, dim=0)[token_i]
                sentence1_embedd.append(vec)
                token_index += 1
                break
            else:
                # Check if tokenized subword is part of the main word
                if tokenized_text[token_i] in search_word:
                    index_increment = 1
                    word_vecs_list = list()
                    base_word = tokenized_text[token_i]
                    
                    # Sum 4 last layers
                    word_vecs_list.append(torch.sum(hidden_states_last_4, dim=0)[token_i]) 
                    while True:
                        composed_word = base_word + tokenized_text[token_i + index_increment].replace('##', '')
                        if composed_word not in search_word:
                            break
                            
                        # Sum 4 last layers
                        word_vecs_list.append(torch.sum(hidden_states_last_4, dim=0)[token_i + index_increment])
                        if composed_word == search_word:
                            avg_vector = torch.mean(torch.stack(word_vecs_list), dim=0)
                            sentence1_embedd.append(avg_vector)    
                            token_index += index_increment # additional increment because element with subword was already analyzed here
                            found = True
                            break   
                        else:
                            base_word = composed_word
                            index_increment += 1                        
            token_index += 1
            if found:
                break

    if len(sentence1_embedd) != len(cleaned_sentence_1):
        raise ValueError('Error in BERT embeddings (probably because of subword embedding, i.e., out-of-vocab words): [%s]' % ', '.join(map(str, tokenized_text)))
    return sentence1_embedd

In [16]:
# Implementation of the WMD metric adapted from:
# https://github.com/RaRe-Technologies/gensim

def wmdistance(bert_model, bert_embeddings_dict, document1, index1, document2, index2):
                
    # If pyemd C extension is available, import it.
    # If pyemd is attempted to be used, but isn't installed, ImportError will be raised in wmdistance
    from pyemd import emd    
    bert_embeddings_doc1 = bert_embeddings_dict[index1]
    bert_embeddings_doc2 = bert_embeddings_dict[index2]

    # Build dict of embeddings
    bert_doc1_dict = {}
    index1 = 0
    for word in document1:
        if word in bert_doc1_dict:
            bert_doc1_dict[word] = torch.mean(torch.stack([bert_doc1_dict[word] , bert_embeddings_doc1[index1]]), dim=0)
        else:
            bert_doc1_dict[word] = bert_embeddings_doc1[index1]
        index1 += 1
        
    # Convert torch tensors to numpy arrays
    for word in bert_doc1_dict:
        bert_doc1_dict[word] = bert_doc1_dict[word].cpu().detach().numpy()

    # Build dict of embeddings
    bert_doc2_dict = {}
    index2 = 0
    for word in document2:
        if word in bert_doc2_dict:
            bert_doc2_dict[word] = torch.mean(torch.stack([bert_doc2_dict[word] , bert_embeddings_doc2[index2]]), dim=0)
        else:
            bert_doc2_dict[word] = bert_embeddings_doc2[index2]
        index2 += 1
        
    # Convert torch tensors to numpy arrays
    for word in bert_doc2_dict:
        bert_doc2_dict[word] = bert_doc2_dict[word].cpu().detach().numpy()

    dictionary = Dictionary(documents=[document1, document2])
    vocab_len = len(dictionary)

    if vocab_len == 1:
        # Both documents are composed by a single unique token
        return 0.0

    # Sets for faster look-up
    docset1 = set(document1)
    docset2 = set(document2)

    # Compute distance matrix
    distance_matrix = np.zeros((vocab_len, vocab_len), dtype=np.double)
    for i, t1 in dictionary.items():
        if t1 not in docset1:
            continue

        for j, t2 in dictionary.items():
            if t2 not in docset2 or distance_matrix[i, j] != 0.0:
                continue

            # Compute Euclidean distance between word vectors
            distance_matrix[i, j] = distance_matrix[j, i] = np.sqrt(np.sum((bert_doc1_dict[t1] - bert_doc2_dict[t2])**2))

    if np.sum(distance_matrix) == 0.0:
        # `emd` gets stuck if the distance matrix contains only zeros.
        logger.info('The distance matrix is all zeros. Aborting (returning inf).')
        return float('inf')

    def nbow(document):
        d = np.zeros(vocab_len, dtype=np.double)
        nbow = dictionary.doc2bow(document)  # Word frequencies
        doc_len = len(document)
        for idx, freq in nbow:
            d[idx] = freq / float(doc_len)  # Normalized word frequencies
        return d

    # Compute nBOW representation of documents
    d1 = nbow(document1)
    d2 = nbow(document2)

    # Compute WMD
    return emd(d1, d2, distance_matrix)

### Initialize BERT model

In [17]:
bert = bert_model_init()
bert_tokenizer = bert[0]
bert_model = bert[1]

In [18]:
# save model
path_save_model = "appr_2_my_model.model"
torch.save(bert_model, path_save_model)

### Extract BERT embeddings for all sentences and store in a read-only data structure

In [19]:
# Build tuples with (step_id, step_text) - used to retrieve the step ID in the end (after the clustering) - and get only test steps for clustering
step_id_text_tuple_list = list()
test_steps_clustering_list = list()
for index, row in test_steps_df.iterrows():
    step_id = row["Step_ID"]
    step_text = row["Steps"]
    step_id_text_tuple_list.append((step_id,step_text))

    temp_list = list()
    if isinstance(row["Steps"], list):
        for elem in row["Steps"]:
            temp_list.append(elem)
    else:
        if isinstance(row["Steps"], str):
            temp_list.append(row["Steps"])
        
    # Build list of lists of tokens (words)
    test_steps_clustering_list.append(temp_list)
    
print("Length of list of tuples:" , len(step_id_text_tuple_list))
print("Length of list with test steps: " , len(test_steps_clustering_list))

Length of list of tuples: 15668
Length of list with test steps:  15668


In [20]:
# Remove empty steps
index = 0
steps_to_remove = list()
for step in test_steps_clustering_list:
    if len(step) == 0:
        steps_to_remove.append(index)
    index += 1

step_id_text_tuple_list = [step_id_text_tuple_list[index] for index in range(len(step_id_text_tuple_list)) if not index in steps_to_remove]
test_steps_clustering_list = [test_steps_clustering_list[index] for index in range(len(test_steps_clustering_list)) if not index in steps_to_remove]
print("Length of list of tuples:" , len(step_id_text_tuple_list))
print("Length of list with test steps: " , len(test_steps_clustering_list))

Length of list of tuples: 15644
Length of list with test steps:  15644


In [ ]:
# Get embeddings for each sentence
bert_embeddings_dict = {}
for index in range(len(test_steps_clustering_list)):
    sentence = test_steps_clustering_list[index]
    bert_embeddings_dict[index] = get_bert_embedding_sentence_words(bert_model, bert_tokenizer, ' '.join(sentence), sentence)
    if len(sentence) != len(bert_embeddings_dict[index]):
        print("Error with embeddings!")
        break

### Compute distance between all pairs of test steps

In [22]:
# Assign the number of test steps to rows and cols
rows = cols = len(test_steps_clustering_list)
dist_matrix = np.zeros((rows, cols))

In [ ]:
start = time.perf_counter()
print("Computing distances....")
for row in range(rows):
    for col in range(row, cols):
        computed_dist = wmdistance(bert_model, bert_embeddings_dict, test_steps_clustering_list[row], row, test_steps_clustering_list[col], col) 
        dist_matrix[row,col] = dist_matrix[col,row] = computed_dist

finish = time.perf_counter()
print(f'Finished in {round(finish-start,2)} second(s)')

In [ ]:
# # save distance matrix
path_save_dist_matrix = 'appr_2_dist_matrix.txt'
# np.savetxt(path_save_dist_matrix, dist_matrix)

# load distance matrix
path_load_dist_matrix = 'appr_2_dist_matrix.txt'
dist_matrix = np.loadtxt(path_load_dist_matrix)

In [ ]:
# Print matrix's shape and check if it's correct
dist_matrix.shape

### Execute hierarchical agglomerative clustering and K-means

In [30]:
# Function to compute the average word vector of a sentence (test step)
def sent_avg_vector(sent, my_model, pre_trained_model):
    list_word_vectors = list()
    num_words = 0
    for word in sent:
        try:
            list_word_vectors.append(my_model[word])
        except:
            try:
                list_word_vectors.append(pre_trained_model[word])
            except:
                return np.zeros(300)
        num_words += 1
    sum_vectors = sum(list_word_vectors)
    avg_vector = sum_vectors/num_words    
    return avg_vector

In [ ]:
# Compute average word vector
avg_word_sentence_vectors = list()
for sentence in test_steps_clustering_list:
    if len(sentence) > 0:
        avg_word_sentence_vectors.append(sent_avg_vector(sentence, my_model, pre_trained_model))  
print("========================")
print("Number of test steps: " , len(avg_word_sentence_vectors))
print("Dimension of one vector: " , len(avg_word_sentence_vectors[0]))

#### Load ground truth of similar test steps (to compute F-score)

In [34]:
# Read excel files with ground truth of similar test steps (cannot be released due to sensitive information)
ground_truth_dir = 'sample_manual_ground_truth/clusters/'
sample_files = os.listdir(ground_truth_dir)

In [35]:
manual_clusters_dict = {}
for sample in sample_files:
    sample_df = pd.read_excel(manual_sample_dir + sample)
    for index, row in sample_df.iterrows():
        cluster_id = row['cluster_id']
        step_id = row['step_id']
        if step_id in manual_clusters_dict:
            existing_list = manual_clusters_dict[step_id]
            existing_list.append(cluster_id)
            manual_clusters_dict[step_id] = existing_list
        else:
            manual_clusters_dict[step_id] = [cluster_id]

In [ ]:
print("Number of test step samples which were manually clustered: ", len(manual_clusters_dict))

In [37]:
test_steps_to_evaluate_list = list(manual_clusters_dict.keys())

In [ ]:
# Use F-score to tune number of clusters and evaluation
aggl_model_list = list()
kmeans_model_list = list()
f_score_aggl_list = list()
f_score_kmeans_list = list()

start = time.time()

# Iterate from 50 to 15,000 clusters
for num_clusters in range(50, 15001, 50):
    start = time.time()
    print("Running for: " + str(num_clusters) + " clusters")
    aggl_clustering_model = AgglomerativeClustering(n_clusters=num_clusters, affinity='euclidean', linkage='average', memory=Memory('/tmp/memory_cache'))
    aggl_clustering_model.fit(dist_matrix)
    aggl_model_list.append(aggl_clustering_model)
    labels = aggl_clustering_model.labels_
    
    # Compute f-score - declare and initialize variables to compute F-score
    TP = 0
    FP = 0
    TN = 0
    FN = 0

    appr_clusters_dict = {}
    for single_label in set(labels):
        indices_label = np.where(labels == single_label)[0].tolist()
        for ind in indices_label:
            appr_clusters_dict[int(step_id_text_tuple_list[ind][0])] = single_label
        
    # Iterate through list of test steps to evaluate
    for i in range(len(test_steps_to_evaluate_list)-1):
        for j in range(i+1, len(test_steps_to_evaluate_list)):
            step_id_1 = test_steps_to_evaluate_list[i]
            step_id_2 = test_steps_to_evaluate_list[j]

            # true positive case
            if (manual_clusters_dict[step_id_1] == manual_clusters_dict[step_id_2]) and (appr_clusters_dict[step_id_1] == appr_clusters_dict[step_id_2]):
                TP += 1

            # false positive case
            elif (manual_clusters_dict[step_id_1] != manual_clusters_dict[step_id_2]) and (appr_clusters_dict[step_id_1] == appr_clusters_dict[step_id_2]):
                FP += 1

            # false negative case
            elif (manual_clusters_dict[step_id_1] == manual_clusters_dict[step_id_2]) and (appr_clusters_dict[step_id_1] != appr_clusters_dict[step_id_2]):
                FN += 1

            # true negative case
            elif (manual_clusters_dict[step_id_1] != manual_clusters_dict[step_id_2]) and (appr_clusters_dict[step_id_1] != appr_clusters_dict[step_id_2]):
                TN += 1
    
    precision = TP / (TP + FP)
    recall = TP / (TP + FN)
    f_score = (2 * precision * recall) / (precision + recall)
    f_score_aggl_list.append(f_score)
    
    # Declare and initialize numpy ndarray for k means centroid initialization with shape: [n_clusters, n_features]
    init_centroid_kmeans = np.zeros((num_clusters,768))

    for single_label in set(labels):
        indices_label = np.where(labels == single_label)[0].tolist()
        sent_vectors_list = [avg_word_sentence_vectors[ind] for ind in indices_label]

        # Compute average sentence vector between the selected sentences in 'sent_vectors_list'
        avg_sent_vectors = sum(sent_vectors_list)/len(sent_vectors_list)
        init_centroid_kmeans[single_label,:] = avg_sent_vectors    
    
    # Run K-Means with the computed centroids
    kmeans = KMeans(n_clusters=num_clusters, init=init_centroid_kmeans, max_iter=300)
    kmeans.fit(avg_word_sentence_vectors)
    kmeans_model_list.append(kmeans)
    labels = kmeans.labels_
    
    # Compute f-score - declare and initialize variables to compute F-score
    TP = 0
    FP = 0
    TN = 0
    FN = 0

    appr_clusters_dict = {}
    for single_label in set(labels):
        indices_label = np.where(labels == single_label)[0].tolist()
        for ind in indices_label:
            appr_clusters_dict[int(step_id_text_tuple_list[ind][0])] = single_label
        
    # Tterate through list of steps to evaluate
    for i in range(len(test_steps_to_evaluate_list)-1):
        for j in range(i+1, len(test_steps_to_evaluate_list)):
            step_id_1 = test_steps_to_evaluate_list[i]
            step_id_2 = test_steps_to_evaluate_list[j]

            # true positive case
            if (manual_clusters_dict[step_id_1] == manual_clusters_dict[step_id_2]) and (appr_clusters_dict[step_id_1] == appr_clusters_dict[step_id_2]):
                TP += 1

            # false positive case
            elif (manual_clusters_dict[step_id_1] != manual_clusters_dict[step_id_2]) and (appr_clusters_dict[step_id_1] == appr_clusters_dict[step_id_2]):
                FP += 1

            # false negative case
            elif (manual_clusters_dict[step_id_1] == manual_clusters_dict[step_id_2]) and (appr_clusters_dict[step_id_1] != appr_clusters_dict[step_id_2]):
                FN += 1

            # true negative case
            elif (manual_clusters_dict[step_id_1] != manual_clusters_dict[step_id_2]) and (appr_clusters_dict[step_id_1] != appr_clusters_dict[step_id_2]):
                TN += 1
    
    precision = TP / (TP + FP)
    recall = TP / (TP + FN)
    f_score = (2 * precision * recall) / (precision + recall)
    f_score_kmeans_list.append(f_score)    
    
    # Delete unnecessary variables to free memory up
    del aggl_clustering_model
    del kmeans
    
    end = time.time()
    print(f'Finished in {round(end-start,2)} second(s)')

### Analysis of hierarhical agglomerative clustering

In [ ]:
# Plot F-score against number of clusters
plt.plot(list(range(50, 15001, 50)), f_score_aggl_list)
plt.title('F-score - Agglomerative')
plt.xlabel('Number of clusters')
plt.ylabel('F-score')
plt.show()

In [ ]:
# Find max F-score and corresponding index
max_score = max(f_score_aggl_list)
max_index = f_score_aggl_list.index(max_score)
print("Best F-score: " , max_score)
print("Index of best F-score: ", max_index)

In [ ]:
best_aggl_model = aggl_model_list[max_index]

In [ ]:
# Get number of clusters of the best model
best_aggl_model_num_clusters = best_aggl_model.n_clusters
best_aggl_model_labels = best_aggl_model.labels_
print("Number of clusters of the best agglomerative model: ", best_aggl_model_num_clusters)

In [ ]:
# save clustered data
path_to_save_data = "appr_2_clustered_data.txt"
out_cluster_file = open(path_to_save_data, "a")
for single_label in set(best_aggl_model_labels):
    indices_label = np.where(best_aggl_model_labels == single_label)[0].tolist()
    for index in indices_label:
        str_to_save = "[" + str(single_label) + "]:\t\t" + test_steps_df.loc[index]["Key"] + "\t\t" + str(step_id_text_tuple_list[index][0]) + "\t\t"+ str(test_steps_clustering_list[index]) + "\n"
        out_cluster_file.write(str_to_save)
out_cluster_file.close()

In [ ]:
# save cluster labels (step IDs)
path_to_save_labels = "appr_2_cluster_labels.txt"
out_cluster_file = open(path_to_save_labels, "a")
for single_label in set(best_aggl_model_labels):
    indices_label = np.where(best_aggl_model_labels == single_label)[0].tolist()
    str_to_save = "[" + str(single_label) + "]: " + ','.join(str(step_id_text_tuple_list[x][0]) for x in indices_label) + "\n"
    out_cluster_file.write(str_to_save)
out_cluster_file.close()

In [ ]:
# save list of models and F-score
model_index = 0
aggl_models_path = "results_approach_2/agglomerative_models/"
for aggl_model in aggl_model_list:
    pickle.dump(aggl_model, open(aggl_models_path + "agglomerative_model_" + str(model_index) + ".pkl", "wb"))
    model_index += 1
2
f_score_list_path = "results_approach_2/appr_2_f_scores.txt"
np.savetxt(f_score_list_path, np.asarray(f_score_aggl_list))

### Analysis of K-means clustering

In [ ]:
# Plot F-score against number of clusters
plt.plot(list(range(50, 15001, 50)), f_score_kmeans_list)
plt.title('F-score - K-Means')
plt.xlabel('Number of clusters')
plt.ylabel('F-score')
plt.show()

In [ ]:
# Find max F-score and corresponding index
max_score = max(f_score_kmeans_list)
max_index = f_score_kmeans_list.index(max_score)
print("Best F-score: " , max_score)
print("Index of best F-score: ", max_index)

In [ ]:
best_kmeans_model = kmeans_model_list[max_index]

In [ ]:
# Get number of clusters of the best model
best_kmeans_model_num_clusters = best_kmeans_model.n_clusters
best_kmeans_model_labels = best_kmeans_model.labels_
print("Number of clusters of the best k-means model: ", best_kmeans_model_num_clusters)
print("Number of non-empty clusters of the best k-means model: ", len(set(best_kmeans_model.labels_)))

In [ ]:
# save clustered data
path_to_save_data = "results_approach_2_kmeans/appr_2_kmeans_clustered_data.txt"
out_cluster_file = open(path_to_save_data, "a")
for single_label in set(best_kmeans_model_labels):
    indices_label = np.where(best_kmeans_model_labels == single_label)[0].tolist()
    for index in indices_label:
        str_to_save = "[" + str(single_label) + "]:\t\t" + test_steps_df.loc[index]["Key"] + "\t\t" + str(step_id_text_tuple_list[index][0]) + "\t\t"+ str(test_steps_clustering_list[index]) + "\n"
        out_cluster_file.write(str_to_save)
out_cluster_file.close()

In [ ]:
# save cluster labels (step IDs)
path_to_save_labels = "appr_2_kmeans_cluster_labels.txt"
out_cluster_file = open(path_to_save_labels, "a")
for single_label in set(best_kmeans_model_labels):
    indices_label = np.where(best_kmeans_model_labels == single_label)[0].tolist()
    str_to_save = "[" + str(single_label) + "]: " + ','.join(str(step_id_text_tuple_list[x][0]) for x in indices_label) + "\n"
    out_cluster_file.write(str_to_save)
out_cluster_file.close()

In [ ]:
# save list of models and F-score
model_index = 0
kmeans_models_path = "results_approach_2/kmeans_models/"
for kmeans_model in kmeans_model_list:
    pickle.dump(kmeans_model, open(kmeans_models_path + "kmeans_model_" + str(model_index) + ".pkl", "wb"))
    model_index += 1

f_score_list_path = "results_approach_2/appr_2_kmeans_f_scores.txt"
np.savetxt(f_score_list_path, np.asarray(f_score_kmeans_list))